## Executing EO4Atlantic Services

EO4Atlantic services may be executed locally outside of the [platform](http://portal.eo4a.science). This is currently achieved by means of Jupyter notebooks, which leverage eo4a.development module functions provided by the service framework, **and is the [recommended method for local service development prior to deployment in the platform](http://docs.eo4a.science/service_development/iterative.html#overview).**

This notebook demonstrates how to execute the example services provided in the [EO4Atlantic service development repository](https://github.com/parameterspace-ie/eo4a-service-development), and may be used as a starting point for executing other (new) services.

In [1]:
# Required imports from service development module.
from eo4a.development import create_wps, describe_service, list_services, monitor_execution, reload_services

As described in the [user guide](http://docs.eo4a.science/service_development/overview.html#service-integration), each EO4Atlantic service is hosted as an [OGC Web Processing Service (WPS)][wps]. When creating the development [service container](http://docs.eo4a.science/service_development/components.html#service-container), any [WPS][wps] definitions found in `$EO4A_SERVICES_DIR` will be made available. The following demonstrates how to list the available services.

[wps]: http://www.opengeospatial.org/standards/wps

In [2]:
wps = create_wps() # This reference is instantiated first, and used in most subsequent calls.
reload_services(wps) # This can be used to reload services when local code changes are made.
list_services(wps)

Processes reloaded successfully:
	merge_shapefiles
	sentinel2-rgb
	gdalinfo
	gdalwarp

Errors:


########## Services: ###########

Merge Shapefiles (id: merge_shapefiles)
----------------------------------------
Merge multiple shapefiles into a single file.


Sentinel-2 RGB (id: sentinel2-rgb)
----------------------------------------
Example service that generates composite RGB rasters from Sentinel-2 products, using bands 4, 3, and 2 respectively by default.
However, alternative bands may also be used, such as the false color (R=8, G=4, B=3).


gdalinfo (id: gdalinfo)
----------------------------------------
Lists information about a raster dataset. See <a href="http://gdal.org/gdalinfo.html" target="_blank">gdalinfo manual</a>.


gdalwarp (id: gdalwarp)
----------------------------------------
Image reprojection and warping utility. See <a href="http://gdal.org/gdalwarp.html" target="_blank">gdalwarp manual</a>.




Full details of individual services, including their [WPS][wps] input and output parameters, can be achieved as follows (the details are loaded from the corresponding `wps.py` module), using the **id** value returned by `list_services()`:

[wps]: http://www.opengeospatial.org/standards/wps

In [3]:
describe_service('sentinel2-rgb', wps) 

Sentinel-2 RGB (id: sentinel2-rgb)
----------------------------------------
Example service that generates composite RGB rasters from Sentinel-2 products, using bands 4, 3, and 2 respectively by default.
However, alternative bands may also be used, such as the false color (R=8, G=4, B=3).


---- Inputs ----
 identifier=s2_product_dir, title=Sentinel-2 product directory, abstract=Contains one or more Sentinel-2 products., data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1


 identifier=r_band, title=R band, abstract=R band number (default=4).,, data type=integer
 Any value allowed
 Default Value: 4 
 minOccurs=0, maxOccurs=1


 identifier=g_band, title=G band, abstract=G band number (default=3).,, data type=integer
 Any value allowed
 Default Value: 3 
 minOccurs=0, maxOccurs=1


 identifier=b_band, title=B band, abstract=B band number (default=2).,, data type=integer
 Any value allowed
 Default Value: 2 
 minOccurs=0, maxOccurs=1


 identifier=resolutio

### Synchronous service execution

Services are typically executed as follows:

1. Prepare a list of WPS inputs (identifier, value) tuples, using the identifiers found in the `describe_service()` output (taken from the corresponding `wps.py` definition).
1. Execute the process with the inputs.
1. Parse the process WPS outputs.

#### Example 1

The first example uses the `gdalinfo` service to retrieve information about a test raster. First, get the list of service inputs:

In [4]:
describe_service('gdalinfo', wps)

gdalinfo (id: gdalinfo)
----------------------------------------
Lists information about a raster dataset. See <a href="http://gdal.org/gdalinfo.html" target="_blank">gdalinfo manual</a>.


---- Inputs ----
 identifier=datasetname, title=Input dataset path, abstract=Full path to input dataset., data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1


 identifier=json, title=Display the output in json format, abstract=Display the output in json format., data type=boolean
 Any value allowed
 Default Value: None 
 minOccurs=0, maxOccurs=1


 identifier=mm, title=Min/max values, abstract=Force computation of the actual min/max values for each band in the dataset., data type=boolean
 Any value allowed
 Default Value: None 
 minOccurs=0, maxOccurs=1


 identifier=stats, title=Image statistics, abstract=Read and display image statistics. Force computation if no statistics are stored in an image., data type=boolean
 Any value allowed
 Default Value: None 
 minOccur

Next, prepare the required inputs, execute the service, and parse the outputs.

In [5]:
inputs = [
    ('datasetname', '/opt/eo4a/services/gdaltools/tests/data/raster_1k_1k.tif'),
    ('json', 'false'),
    ('mm', 'true'),
    ('stats', 'true'),
    ('approx_stats', 'true'),
    ('hist', 'true'),
    ('nogcp', 'true'),
    ('nomd', 'false'),
    ('norat', 'true'),
    ('noct', 'true'),
    ('checksum', 'true'),
    ('listmdd', 'true'),
    ('mdd', 'all'),
    ('nofl', 'true'),
    ('proj4', 'true'),
]
# Synchronous execution
execution = wps.execute('gdalinfo', inputs)

# Check the description for the outputs provided by the service
# Only one output for this service
print('gdalinfo output:')
output = execution.processOutputs[0]
print('%s: %s' % (output.identifier, output.data[0]))

gdalinfo output:
output: Driver: GTiff/GeoTIFF
Files: /opt/eo4a/services/gdaltools/tests/data/raster_1k_1k.tif
Size is 110, 110
Coordinate System is:
PROJCS["WGS 84 / UTM zone 29N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-9],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32629"]]
PROJ.4 string is:
'+proj=utm +zone=29 +datum=WGS84 +units=m +no_defs '
Origin = (60

#### Example 2

The second example uses the `gdalwarp` service to modify the resolution of a test raster. As before, get the list of service inputs (Note - only a selection of the possible `gdalwarp` tool parameters are currently available):

In [6]:
describe_service('gdalwarp', wps)

gdalwarp (id: gdalwarp)
----------------------------------------
Image reprojection and warping utility. See <a href="http://gdal.org/gdalwarp.html" target="_blank">gdalwarp manual</a>.


---- Inputs ----
 identifier=srcfile, title=Source file name, abstract=Full path to source file name., data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1


 identifier=dstfile, title=Destination file name, abstract=Full path to destination file name., data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1


 identifier=tr, title=Output file resolution, abstract=Set output file resolution (in target georeferenced units). 
                -tr xres yres, data type=string
 Any value allowed
 Default Value: None 
 minOccurs=0, maxOccurs=1


 identifier=co, title="NAME=VALUE", abstract=Passes a creation option to the output format driver. 
                Multiple -co options may be listed. 
                See format specific documentation for l

Again, prepare the required inputs, execute the service, and parse the outputs.

In [7]:
inputs = [
    ('srcfile', '/opt/eo4a/services/gdaltools/tests/data/raster_1k_1k.tif'),
    ('dstfile', '/tmp/raster_2k_2k.tif'),
    ('tr', '2000 2000'),
    ('co', 'COMPRESS=LZW'),    
    ('overwrite', 'true'),
]
# Synchronous execution
execution = wps.execute('gdalwarp', inputs)

# Check the description for the outputs provided by the service
# Only one output for this service
print('gdalwarp output:')
output = execution.processOutputs[0]
output_raster = output.data[0]
print('%s: %s' % (output.identifier, output_raster))

gdalwarp output:
dstfile: /tmp/raster_2k_2k.tif


The resolution of this file can be checked with the `gdalinfo` service (check the `Pixel Size` values in the output):

In [8]:
inputs = [
    ('datasetname', output_raster),
    ('json', 'false'),
    ('mm', 'true'),
    ('stats', 'true'),
    ('approx_stats', 'true'),
    ('hist', 'true'),
    ('nogcp', 'true'),
    ('nomd', 'false'),
    ('norat', 'true'),
    ('noct', 'true'),
    ('checksum', 'true'),
    ('listmdd', 'true'),
    ('mdd', 'all'),
    ('nofl', 'true'),
    ('proj4', 'true'),
]
# Synchronous execution
execution = wps.execute('gdalinfo', inputs)

# Check the description for the outputs provided by the service
# Only one output for this service
print('gdalinfo output:')
output = execution.processOutputs[0]
print('%s: %s' % (output.identifier, output.data[0]))

gdalinfo output:
output: Driver: GTiff/GeoTIFF
Files: /tmp/raster_2k_2k.tif
Size is 55, 55
Coordinate System is:
PROJCS["WGS 84 / UTM zone 29N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-9],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32629"]]
PROJ.4 string is:
'+proj=utm +zone=29 +datum=WGS84 +units=m +no_defs '
Origin = (600000.000000000000000,6000000.00000000

### Asynchronous service execution

It is also possible to execute services asynchronously, where this is recommended for services that may be time-consuming. *Note: services are always executed asynchronously when hosted in the EO4Atlantic platform.* 
    
The process is similar to synchronous execution:

1. Prepare a list of WPS inputs (identifier, value) tuples, using the identifiers found in the `describe_service()` output (taken from the corresponding `wps.py` definition).
1. Asynchronously execute the process with the inputs.
  1. This is achieved by specifying an optional identifier specifying that process output is to be returned as a reference rather than in the response from the call to `execute()`.
1. Monitor the execution progress.
1. Upon completion, parse the process WPS outputs.

#### Example 3

The next example uses the `sentinel2-rgb` service to generate a composite RGB raster for a particular Sentinel-2 product. It is executed asynchronously. It assumes that a test product has been previously copied to the host data volume (`$EO4A_DATA_DIR/service/test_s2products`). **Using directories in `$EO4A_DATA_DIR/service` is the recommended approach for local execution of services using real products.**

In [9]:
inputs = [
    ('s2_product_dir', 'test_s2products'), # Location relative to $EO4A_DATA_DIR/service on the host
    ('r_band', '4'),
    ('g_band', '3'),
    ('b_band', '2'),
    ('resolution', '60'),
]
# Aynchronous execution - note the specification of the 'output' reference
execution = wps.execute('sentinel2-rgb', inputs,  output='output_reference')

monitor_execution(execution, sleep_secs=5) # Check every 3 seconds by default
execution.getOutput()

# Check the description for the outputs provided by the service
# Only one output for this service
print('RGB output is available at (relative to $EO4A_DATA_DIR/service):')
output = execution.processOutputs[0]
print('%s: %s' % (output.identifier, output.data[0]))

Status(0% - ProcessAccepted) PyWPS Process sentinel2-rgb accepted
Status(20% - ProcessStarted) Executing process.
Status(20% - ProcessSucceeded) Finished
Status: ProcessSucceeded
RGB output is available at (relative to $EO4A_DATA_DIR/service):
rgb_dir: sentinel2-rgb/output/rgb
